# Install and import library

In [1]:
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.models import Sequential
from keras.applications import VGG16
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Preprocessing

In [ ]:
# Generator data gambar dengan rescaling
image_generator = ImageDataGenerator(
    rescale=1./255, 
)
 
# Tentukan ukuran gambar
size_w = 150
size_h = size_w
size = (size_w, size_h)

batch = 32
 
# Tentukan jalur direktori untuk data latih, validasi, dan uji
train_dir = '../TA/original_dataset/train'
val_dir = '../TA/original_dataset/val'
test_dir = '../TA/original_dataset/test'

# Alirkan data dari direktori untuk data latih
train_data = image_generator.flow_from_directory(
    train_dir,
    target_size=size,
    batch_size=batch,
    class_mode='categorical',
    color_mode='rgb'
)

# Alirkan data dari direktori untuk data validasi
validation_data = image_generator.flow_from_directory(
    val_dir,
    target_size=size,
    batch_size=batch,
    class_mode='categorical',
    color_mode='rgb'
)

# Alirkan data dari direktori untuk data uji
test_data = image_generator.flow_from_directory(
    test_dir,
    target_size=size,
    batch_size=batch,
    class_mode='categorical',
    color_mode='rgb'
)

In [ ]:
def switch_dict_key_values(this_dict):
    return dict((v,k) for k,v in this_dict.items())

classes_name = switch_dict_key_values(train_data.class_indices)
print(classes_name)

# Model

In [ ]:
# Load pre-trained VGG16 model
base_model = VGG16(weights=None, include_top=False, input_shape=(size_w, size_h, 3))
 
# Create a new model on top of the pre-trained VGG16
model = Sequential()
model.add(base_model)

# Add Batch Normalization for data normalization
#model.add(BatchNormalization())

# Add additional convolutional layers with weight decay
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2, padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2, padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2, padding='same'))
model.add(Dropout(0.5))

# Flatten and add fully connected layers with Dropout
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(1e-4)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output layer
model.add(Dense(20, activation='softmax'))

# Menampilkan ringkasan model
model.summary()

In [ ]:
# Kompilasi model dengan optimizer Adam, menggunakan gradient clipping
opt = Adam(learning_rate=0.0001, clipnorm=1.0)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Definisi callback EarlyStopping untuk mencegah overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
# Train the model
epochs = 5
history = model.fit(
    train_data, 
    epochs=epochs,
    batch_size = 64,
    validation_data=validation_data, 
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_data)
print(f'Test accuracy: {test_acc}')